In [19]:
# We will do the imports required for this notebook here
%load_ext autoreload
%autoreload 2
# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm

# swift is a lightweight browser-based simulator which comes eith the toolbox
from swift import Swift

# spatialgeometry is a utility package for dealing with geometric objects
import spatialgeometry as sg

# import urchin to read urdf files
from urchin import URDF

# this package provides several solvers for solving quadratic programmes
import qpsolvers as qp 

# modern robotics code library accompanying 
# Modern Robotics: Mechanics, Planning, and Control (Kevin Lynch and Frank Park)
import modern_robotics as mr

# the os package provides utilities for interacting with the operating system
import os

# import config 
from config.config import * 
from models.panda_model import *
from planning.screw import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


planning.screw.Screw

In [75]:
screw = Screw(theta=np.pi/2, d=0, u=np.array([0, 0, 1]), r=np.array([0, 0, 0]))
dquat = screw.dquat()

print(dquat.conj())

 0.7071 < -0.0000, -0.0000, -0.7071 > + ε  0.0000 < -0.0000, -0.0000, -0.0000 >


In [73]:
dualq = sm.UnitDualQuaternion(sm.SE3([-0.5, 0.5, 0.5]))
dualq2 = sm.UnitDualQuaternion(sm.SE3([1, 2, 3]))

sclerp = sclerp(dualq, dualq2, 0.5)

AttributeError: 'UnitQuaternion' object has no attribute 'x'

In [ ]:
def init_env():
    # Make the environment
    env = Swift()
    env.launch(realtime=True, browser="notebook")
    
    # import robot
    panda_virtual = create_virtual_panda()
    # set the robot config to ready position
    panda_virtual.q = panda_virtual.qr
    # open the fingers
    panda_virtual.grippers[0].q = [0.035, 0.035]
    
    # create object to grasp
    box = sg.Cuboid(scale=[0.1, 0.07, 0.03],
                    color='blue')
    box.T = sm.SE3(0.7, 0, 0.015)
    box.set_alpha(0.5)
    

    # Set a desired and effector pose an an offset from the current end-effector pose
    bTe = sm.SE3(-box.scale[0]/2 + 0.01, 0, 0) * sm.SE3.Rx(np.pi/2)
    Tep = box.T * bTe
    
    
    # end-effector axes
    ee_axes = sg.Axes(0.1)
    ee_axes.T = panda_virtual.fkine(panda_virtual.q, end='panda_finger_virtual')

    # goal axes
    goal_axes = sg.Axes(0.1)
    # Set the goal axes to Tep
    goal_axes.T = Tep
    
    # add to environment
    
    env.add(panda_virtual)
    env.add(box)
    env.add(ee_axes)
    env.add(goal_axes)
    
    return env, Tep, panda_virtual, ee_axes, goal_axes


In [ ]:
env, Tep, panda_virtual, ee_axes, goal_axes = init_env()

# Simulate the robot while it has not arrived at the goal
arrived = False
while not arrived:

    # Work out the required end-effector velocity to go towards the goal
    Te = panda_virtual.fkine(panda_virtual.q, end='panda_finger_virtual')
    ee_axes.T = Te
    v, arrived = rtb.p_servo(Te, Tep, threshold=0.001)
    
    # Set the Panda's joint velocities
    panda_virtual.qd = np.linalg.pinv(panda_virtual.jacobe(panda_virtual.q, end='panda_finger_virtual')) @ v
    
    # Step the simulator by 50 milliseconds
    env.step(0.05)


In [113]:
from utils.ik_jl import *

ets = panda_virtual.ets(end='panda_finger_virtual')
iksolver = IK_JL(ets, k0=[2]*7 + [0])

iksol = iksolver.solve(ets=ets, Tep=Tep)

iksol

IKSolution(q=array([-0.94249315,  1.15665915,  0.88906795, -1.90294011, -2.83780795,
        2.58991226,  2.77861967, -0.93748292]), success=True, iterations=633, searches=25, residual=8.826470250498193e-07, reason='Success')

In [165]:
def secondary_object(robot: rtb.Robot, q: np.ndarray) -> np.ndarray:
    """
    Calculate the gradient of the joint limit avoidance function

    Parameters
    ----------
    robot
        The robot the manipulators kinematics
    q
        The current joint coordinate vector

    Returns
    -------
    dw_dq
        The gradient of the joint limit avoidance function
    """
    n = robot.n
    qlim = robot.qlim
    
    q_mid = (qlim[1] + qlim[0]) / 2
    q_range = qlim[1] - qlim[0]
    
    
    # Vectorized calculation with singularity protection
    with np.errstate(divide='ignore', invalid='ignore'):
        dw_dq = -1/n * (q - q_mid) / (q_range)**2
        
    # Handle unlimited joints (range = 0)
    dw_dq[np.isinf(dw_dq)] = 0  
    dw_dq[np.isnan(dw_dq)] = 0
    
    return dw_dq

In [208]:
env, Tep, panda_virtual, ee_axes, goal_axes = init_env()
dt = 0.05
arrived = False
# Specify the gain for the p_servo method
kt = 1.5
kr = 1.0
k = np.array([kt, kt, kt, kr, kr, kr])
n = panda_virtual.n

# gain for the joint limit avoidance
k0 = 1


Vep = np.zeros(6)

# Run the simulation until the robot arrives at the goal
while not arrived:

    # Work out the base frame manipulator Jacobian using the current robot configuration
    J = panda_virtual.jacobe(panda_virtual.q, end='panda_finger_virtual')
    J_pinv = np.linalg.pinv(J)

    # The end-effector pose of the panda (using .A to get a numpy array instead of an SE3 object)
    Te = panda_virtual.fkine(panda_virtual.q, end='panda_finger_virtual').A

    # Calculate the required end-effector velocity and whether the robot has arrived
    ev, arrived = rtb.p_servo(Te, Tep, gain=k, threshold=0.001, method="twist")


    dw_dq = secondary_object(panda_virtual, panda_virtual.q)
    
    q0 = k0 * dw_dq
    qd = J_pinv @ (Vep + ev) + (np.eye(n) - J_pinv @ J) @ q0
    
    # Apply the joint velocities to the Panda
    panda_virtual.qd[:n] = qd[:n]

    # Update the ee axes
    ee_axes.T = Te

    # Step the simulator by dt seconds
    env.step(dt)

In [204]:
Te, Tep

(array([[ 9.99999559e-01, -9.39403932e-04,  1.95064736e-07,
          6.59996165e-01],
        [ 1.95278604e-07,  2.27571650e-07, -1.00000000e+00,
          1.63529431e-08],
        [ 9.39403932e-04,  9.99999559e-01,  2.27754995e-07,
          1.50073512e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 array([[ 1.000000e+00,  0.000000e+00,  0.000000e+00,  6.600000e-01],
        [ 0.000000e+00,  6.123234e-17, -1.000000e+00,  0.000000e+00],
        [ 0.000000e+00,  1.000000e+00,  6.123234e-17,  1.500000e-02],
        [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]]))

In [ ]:
ev, arrived = rtb.p_servo(Te, Tep, gain=k, threshold=0.001, method="rpy")

(array([ 6.60004115e-01,  1.14770391e-02,  1.16101651e-02,  1.57079597e+00,
        -7.37664909e-04,  7.37947502e-04]),
 array([[ 0.00000000e+00, -7.37947502e-04, -7.37664909e-04,
          6.60004115e-01],
        [ 7.37947502e-04,  0.00000000e+00, -1.57079597e+00,
          1.14770391e-02],
        [ 7.37664909e-04,  1.57079597e+00,  0.00000000e+00,
          1.16101651e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]]),
 array([ 1.57079597e+00, -7.37664909e-04,  7.37947502e-04,  6.60004115e-01,
         1.14770391e-02,  1.16101651e-02]))

In [ ]:
# import genesis as gs

# gs.init(backend=gs.cuda)

# scene = gs.Scene(
#     show_viewer = True,
#     viewer_options = gs.options.ViewerOptions(
#         res           = (1280, 960),
#         camera_pos    = (3.5, 0.0, 2.5),
#         camera_lookat = (0.0, 0.0, 0.5),
#         camera_fov    = 40,
#         max_FPS       = 60,
#     ),
#     vis_options = gs.options.VisOptions(
#         show_world_frame = True,
#         world_frame_size = 1.0,
#         show_link_frame  = False,
#         show_cameras     = False,
#         plane_reflection = True,
#         ambient_light    = (0.1, 0.1, 0.1),
#     ),
#     renderer=gs.renderers.Rasterizer(),
# )

# plane = scene.add_entity(
#     gs.morphs.Plane(),
# )
# franka = scene.add_entity(
#     gs.morphs.MJCF(file='xml/franka_emika_panda/panda.xml'),
# )

# cam = scene.add_camera(
#     res    = (640, 480),
#     pos    = (3.5, 0.0, 2.5),
#     lookat = (0, 0, 0.5),
#     fov    = 30,
#     GUI    = False,
# )

# scene.build()

# # render rgb, depth, segmentation, and normal
# # rgb, depth, segmentation, normal = cam.render(rgb=True, depth=True, segmentation=True, normal=True)

# cam.start_recording()
# import numpy as np

# for i in range(120):
#     scene.step()
#     cam.set_pose(
#         pos    = (3.0 * np.sin(i / 60), 3.0 * np.cos(i / 60), 2.5),
#         lookat = (0, 0, 0.5),
#     )
#     cam.render()
# cam.stop_recording(save_to_filename='video.mp4', fps=60)

In [ ]:
# import numpy as np
# import genesis as gs

# ########################## init ##########################
# gs.init(backend=gs.cuda)

# ########################## create a scene ##########################
# scene = gs.Scene(
#     show_viewer = True,
#     viewer_options = gs.options.ViewerOptions(
#         res           = (1280, 960),
#         camera_pos    = (3.5, 0.0, 2.5),
#         camera_lookat = (0.0, 0.0, 0.5),
#         camera_fov    = 40,
#         max_FPS       = 60,
#     ),
#     vis_options = gs.options.VisOptions(
#         show_world_frame = True,
#         world_frame_size = 1.0,
#         show_link_frame  = False,
#         show_cameras     = False,
#         plane_reflection = True,
#         ambient_light    = (0.1, 0.1, 0.1),
#     ),
#     renderer=gs.renderers.Rasterizer(),
# )

# cam = scene.add_camera(
#     res    = (640, 480),
#     pos    = (3.5, 0.0, 2.5),
#     lookat = (0, 0, 0.5),
#     fov    = 30,
#     GUI    = False,
# )


# ########################## entities ##########################
# plane = scene.add_entity(
#     gs.morphs.Plane(),
# )
# cube = scene.add_entity(
#     gs.morphs.Box(
#         size = (0.04, 0.04, 0.04),
#         pos  = (0.65, 0.0, 0.02),
#     )
# )
# franka = scene.add_entity(
#     gs.morphs.MJCF(file='xml/franka_emika_panda/panda.xml'),
# )
# ########################## build ##########################
# scene.build()

# motors_dof = np.arange(7)
# fingers_dof = np.arange(7, 9)

# # set control gains
# # Note: the following values are tuned for achieving best behavior with Franka
# # Typically, each new robot would have a different set of parameters.
# # Sometimes high-quality URDF or XML file would also provide this and will be parsed.
# franka.set_dofs_kp(
#     np.array([4500, 4500, 3500, 3500, 2000, 2000, 2000, 100, 100]),
# )
# franka.set_dofs_kv(
#     np.array([450, 450, 350, 350, 200, 200, 200, 10, 10]),
# )
# franka.set_dofs_force_range(
#     np.array([-87, -87, -87, -87, -12, -12, -12, -100, -100]),
#     np.array([ 87,  87,  87,  87,  12,  12,  12,  100,  100]),
# )

In [ ]:

# # get the end-effector link
# end_effector = franka.get_link('hand')

# # move to pre-grasp pose
# qpos = franka.inverse_kinematics(
#     link = end_effector,
#     pos  = np.array([0.65, 0.0, 0.25]),
#     quat = np.array([0, 1, 0, 0]),
# )
# # gripper open pos
# qpos[-2:] = 0.04
# path = franka.plan_path(
#     qpos_goal     = qpos,
#     num_waypoints = 200, # 2s duration
# )
# # execute the planned path
# for waypoint in path:
#     franka.control_dofs_position(waypoint)
#     scene.step()

# # allow robot to reach the last waypoint
# for i in range(100):
#     scene.step()
# # reach
# qpos = franka.inverse_kinematics(
#     link = end_effector,
#     pos  = np.array([0.65, 0.0, 0.130]),
#     quat = np.array([0, 1, 0, 0]),
# )
# franka.control_dofs_position(qpos[:-2], motors_dof)
# for i in range(100):
#     scene.step()

# # grasp
# franka.control_dofs_position(qpos[:-2], motors_dof)
# franka.control_dofs_force(np.array([-0.5, -0.5]), fingers_dof)

# for i in range(100):
#     scene.step()

# # lift
# qpos = franka.inverse_kinematics(
#     link=end_effector,
#     pos=np.array([0.65, 0.0, 0.28]),
#     quat=np.array([0, 1, 0, 0]),
# )
# franka.control_dofs_position(qpos[:-2], motors_dof)
# for i in range(200):
#     scene.step()